### defining the transportation unit class
<li>a transportation unit has a location (latitude and longitude)<li>person capacity: maximum of number of person that can be moved<li>battery_state: what percentage of battery is still full<li>battery_capacity: number of kwh when fully charged<li>avg_km_per_kwh : number of km driver per kilometer<li>state: possible values 1 = 'charging', 2 = 'discharging', 3 = 'stationary' (doing nothing), 4 = 'person transport', 5 = 'package transport

a tranportation unit will constantly make a request/decision about what to do next:
charge, discharge, transport person, transport package, do nothing (or maybe move empty to area with highest chance of picking up person/package)




In [4]:
import ipynb.fs.defs.helper_functions as helper_functions

In [41]:
class Tu():
    def __init__(self, name, lat, lon, person_capacity = 4, battery_state=0.2, battery_capacity=300, 
                 avg_km_per_kwh=7, state=4
                ):
        "assumes name is string"
        self.name = name
        self.lat = lat
        self.lon = lon
        self.person_capacity = person_capacity
        self.battery_state = battery_state
        self.battery_capacity = battery_capacity
        self.avg_km_per_kwh = avg_km_per_kwh
        self.state = state
    
    def get_name(self):
        return self.name
    
    def get_lon(self):
        return self.lon

    def get_lat(self):
        return self.lat

    def get_battery_state(self):
        return self.battery_state
    
    def get_state(self):
        return self.get_state
    
    def get_radius(self):
        return self.battery_capacity * self.battery_state * self.avg_km_per_kwh
    
    def __str__(self):
        return self.get_name() \
        + ' at lat:' + str(self.get_lat()) + ' and lon:' + str(self.get_lon())\
        +' radius left:'  + str(self.get_radius())
        
    def update_location(self, lon, lat):
        self.lon = lon
        self.lat = lat
        
    def update_state(self, state):
        self.state = state
        
#  where should we calculate the decrease in capacity after a trip has been completed
#  is this a method based upon the new location? 
#  where do we calculate the charging of the battery

    def update_capacity(self, capacity):
        self.capacity = capacity

# next action defines what the car will do next      
# maybe we can also use reinforcement learning with multi arm bandit logic where the arms are the possible actions
#     def next_action(self):
#         update state
#         update time
#         update capacity

# I guess a TU will exist in a certain time, how do we 'simulate'  this
#     def update_time(self, time):
#         self.time = time

        
# transportation unit in center of groningen at t = 0:00   
tu1 = Tu(name='tu1', lat=6.5665, lon=53.2194, person_capacity = 4, battery_state=0.2, battery_capacity=85, 
                 avg_km_per_kwh=7, state=4)

print(tu1)
    

tu1 at lat:6.5665 and lon:53.2194 radius left:119.0


In [42]:
# loc1 = location of car
loc1 = (tu1.get_lon(), tu1.get_lat())
# loc2 = location of person requesting transportation
loc2 = (53.2350682,6.6068002)

# print(loc1, loc2)
print('distance to request is', helper_functions.distance ((tu1.get_lon(), tu1.get_lat()), loc2))

# after a ride from center of groningen to outskirt has finished
# the car has travelled 
tu1.update_location(lat=6.6068002, lon=53.2350682)
loc1 = (tu1.get_lon(), tu1.get_lat())
print('distance now is: ', helper_functions.distance(loc1, loc2))

distance to request is 3.19972640745685
distance now is:  0.0


In [30]:
helper_functions.distance((53.2194, 6.5665), (53.2350682, 6.6068002))

3.19972640745685

### transportation_request class

In [44]:
import datetime

class Tr():
    def __init__(self, start_lat, start_lon, end_lat, end_lon,  request_time, transport_type, request_id):
        self.start_lat = start_lat
        self.start_lon = start_lon
        self.end_lat = end_lat
        self.end_lon = end_lon
        self.request_id = request_id
        self.transport_type = transport_type
        self.request_time = request_time
    
    def get_distance_ride(self):
        return helper_functions.distance((self.start_lat, self.start_lon),(self.end_lat,self.end_lon))
    
    def get_request_id(self):
        return self.request_id
    
    def get_distance_to_tu(self, tu_location):
        #location of transportation unit
        return helper_functions.distance((tu_location[0], tu_location[1]), (self.start_lat, self.start_lon))
    
    def __str__(self):
             return 'id:' + str(self.get_request_id()) + ' requests a transport at ' \
        + str(self.request_time) +' for ' \
        + str(round(self.get_distance_ride(),3)) + " KM" \
        #+ datetime.time.strftime("%Y-%m-%d %H:%M:%S", self.request_time)


tr1 = Tr(start_lat=53.2194, start_lon=6.5665, end_lat=53.2350682,  end_lon=6.6068002,     \
         request_time=(2018,4,1,8,25,0), transport_type='p', request_id=1)

print(tr1)
        

id:1 requests a transport at (2018, 4, 1, 8, 25, 0) for 3.2 KM


### iterating over all classses?
https://stackoverflow.com/questions/739882/iterating-over-object-instances-of-a-given-class-in-python
is this a good way to find open requests, all open request could iterate over the available cars and once the request is succesfully fullfilled (in tangle) written to the database

wat moet er centraal staan in aanvraag.
wanneer er b.v. een auto beschikbaar is die nog 10km van aanvrager weg is en er op 500 meter een auto beschikbaar komt over 30 seconde omdat deze dan klaar is met het vervoeren van zijn huidige passagier. wat is dan het beste.

wanneer we de auto centraal zetten gaat deze zoeken naar een passagier, de auto vindt de dichtstbijzijnde passagier die staat te wachten op 10KM, deze twee gaan dan een transactie aan en de auto krijgt de state:'onderweg' en de passagier krijgt de state: 'wordt opgepikt' 

wanneer de passagier centraal staat kijkt deze naar alle auto's,  

auto's zonder passagier: rechtstreekse route

auto's met passagier: indirecte route (huidige locatie van auto naar eindbestemming en van eindbestemming naar passagier)

qua optimalisatie lijkt tweede optie me het handigst 
